In [1]:
from ipywidgets import widgets, Layout
from IPython.display import display
from Crypto.PublicKey import RSA, ECC
from Crypto.Cipher import PKCS1_OAEP, AES
import binascii
import random
import math
from tinyec import registry
import secrets

In [2]:
#credits
#widget info help from https://ipywidgets.readthedocs.io/en/stable/
#styling help from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#RSA help from https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples
#DH help from https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange
#ECC help from https://cryptobook.nakov.com/asymmetric-key-ciphers/ecc-encryption-decryption

In [35]:
#create widgets
intr  ='Anything typed while text is encoded will not load. You can encrypt your message multiple times, \
                but you will have to decrypt in the same order.'
intro = widgets.HTML(value= '<style>p{word-wrap: break-word}</style> <p>'+ intr +' </p>')
style = {'description_width': 'initial'}
keyIn = widgets.Text(placeholder='type here', description='Enter your plaintext:', disabled=False, \
                style=style, layout=Layout(width='400px'))
dropdown = widgets.Dropdown(options=['RSA', 'Diffe-Hellman','Elliptic curve'], value='RSA', 
                description='Choose an encryption method:', disabled=False, style=style, \
                layout=Layout(width='400px'))
encdec = widgets.ToggleButton(description='Encrypt/Decrypt', disabled=False)
label = widgets.Label(value='Output:')
output = widgets.Output(layout=Layout(height='100px', width='100%', overflow='hidden scroll',
                border='1px solid black'))
inputBox = widgets.VBox([intro, keyIn, dropdown, encdec, label, output],\
                layout=widgets.Layout(margin='5px 5px 50px 5px', width="500px"))

#display widgets
display(inputBox)

#create variables
plaintext = keyIn.value
encryptionMode = dropdown.value
ciphertext = plaintext
encrypted = '.'
flag = 'e'
#RSAvariables
RSAkeyPair = 0
RSApublicKey = 0
RSApublicKeyPEM = 0
RSAprivateKeyPEM = 0
#DHvariables
DHp = 23 #assume these are already agreed upon; can be changed
DHg = 5 #primitive root modulo 23
DHAsecretInt = math.floor(random.random() *10)
DHApublicKey = (DHg**DHAsecretInt)%DHp
DHBsecretInt = math.floor(random.random() *10)
DHBpublicKey = (DHg**DHBsecretInt)%DHp
DHAsecret = (DHBpublicKey**DHAsecretInt)%DHp
DHBsecret = (DHApublicKey**DHBsecretInt)%DHp
padd = chr(math.floor(random.random() *10))
#ECC variables
ECcurve = registry.get_curve('brainpoolP256r1') #brainpoolP256r1 curve
ECCciphertextPrivKey = secrets.randbelow(ECcurve.field.n)
ECCciphertextPubKey = ECCciphertextPrivKey * ECcurve.g
ECCprivKey = secrets.randbelow(ECcurve.field.n)
ECCpubKey = ECCprivKey * ECcurve.g
ECCencryptKey = ECCpubKey * ECCciphertextPrivKey
ECCdecryptKey = ECCciphertextPubKey * ECCprivKey

#handlers & functions
def compress(point): #ECC helper
    return hex(point.x) + hex(point.y % 2)[2:]

def RSAkeygen():
    global RSAkeyPair, RSApublicKey, RSApublicKeyPEM, RSAprivateKeyPEM
    RSAkeyPair = RSA.generate(1024) #1024 bit key --> roughly 40 characters; standard is 3072 or 4096
    RSApublicKey = RSAkeyPair.publickey()
    RSApublicKeyPEM = RSApublicKey.exportKey()
    RSAprivateKeyPEM = RSAkeyPair.exportKey()
RSAkeygen()
    
def encr(plain, mode):
    global RSApublicKey, ciphertext, encrypted
    cipher=''
    if (mode == 'RSA'): 
        cipher = bytes(plain, 'utf-8')
        encryptor = PKCS1_OAEP.new(RSApublicKey)
        encrypted = encryptor.encrypt(cipher)
        ciphertext = binascii.b2a_base64(encrypted).decode()
    elif (mode == 'Diffe-Hellman'):
        for i in plain:
            cipher = cipher + chr(ord(i)^ord(padd));
        ciphertext = cipher
    else:
        for i in range(0, len(plain)):
            cipher = cipher + chr(ord(plain[i])^ord(compress(ECCencryptKey)[i]))
        ciphertext=cipher
    with output:
        output.clear_output()
        print(ciphertext)

def decr(cipher, mode):
    global plaintext, encrypted
    plain = ''
    if (mode == 'RSA'):
        decryptor = PKCS1_OAEP.new(RSAkeyPair)
        plaintext = decryptor.decrypt(encrypted).decode()
    elif (mode == 'Diffe-Hellman'):
        for i in cipher:
            plain = plain + chr(ord(i)^ord(padd));
        plaintext = plain
    else:
        for i in range(0, len(cipher)):
            plain = plain + chr(ord(cipher[i])^ord(compress(ECCencryptKey)[i]))
            plaintext = plain
    with output:
        output.clear_output()
        print(plaintext)

def dropdownHandler(change):
    global plaintext, ciphertext, encryptionMode, flag
    plaintext = keyIn.value
    encryptionMode = change.new
dropdown.observe(dropdownHandler, names='value')

def keyInHandler(change):
    global plaintext, ciphertext, encryptionMode, flag
    plaintext = change.new
    encryptionMode = dropdown.value
    with output:
        output.clear_output()
        print(plaintext)    
keyIn.observe(keyInHandler, names='value')

def encdecHandler(obj):
    global plaintext, ciphertext, encryptionMode, flag
    if flag == 'e':
        flag = 'd'
        encr(plaintext, encryptionMode)
    else:
        flag ='e'
        decr(ciphertext, encryptionMode)
encdec.observe(encdecHandler, 'value')